# Sesión 5 - Inteligencia Artificial
## Belén Díaz Agudo -  Facultad de Informática UCM
## Búsqueda local
En esta primera parte usaremos ejercicios paso a paso para familiarizarnos con la resolución de problemas sencillos de optimización, problemas conocidos que vamos a resolver utilizando algoritmos de búsqueda local. 
En la segunda parte de la práctica se pide resolver el problema dado en el enunciado.

## Parte 1. Algoritmo de escalada
Hill Climbing es un algoritmo de búsqueda local heurística utilizada para problemas de optimización.
Esta solución puede o no ser el óptimo global. El algoritmo es una variante del algoritmo de generación y prueba.
<br>
En general, el algoritmo funciona de la siguiente manera:
- Evaluar el estado inicial.
- Si es igual al estado del objetivo, terminamos.
- Encuentra un estado vecino al estado actual
- Evaluar este estado. Si está más cerca del estado objetivo que antes, reemplace el estado inicial con este estado y repita estos pasos.
<br>
Usaremos la implementación de AIMA que está en el módulo search.py

    def hill_climbing(problem):
        """From the initial node, keep choosing the neighbor with highest value,
        stopping when no neighbor is better. [Figure 4.2]"""
        current = Node(problem.initial)
        while True:
            neighbors = current.expand(problem)
            if not neighbors:
                break
            neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
            if problem.value(neighbor.state) <= problem.value(current.state):
                break
            current = neighbor
        return current.state


### TSP (Travelling Salesman Problem): el problema del viajante
Dado un conjunto de ciudades y la distancia entre cada par de ciudades, el problema es encontrar la ruta más corta posible que visite cada ciudad exactamente una vez y regrese al punto de partida. Es un problema NP hard. No existen una solución de coste polinomial. 

In [408]:
cd aima

[Errno 2] No such file or directory: 'aima'
/home/azul42/Escritorio/Universidad/Cuarto - Madrid/PrimerCuatrimestre/IA-1/Pracs/aima


In [409]:
##Resolvereremos el problema del viajante TSP para encontrar una solución aproximada.
from search import *

class TSP_problem(Problem):

    def two_opt(self, state):
        """ Neighbour generating function for Traveling Salesman Problem """
        ## Puedes buscar información adicional del método 2-opt. 
        ## Un vecino 2-opt consiste en eliminar dos aristas y volver a conectar los dos caminos resultantes de 
        ## forma diferente para obtener un nuevo recorrido. 
                
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state) - 1)
        right = random.randint(0, len(neighbour_state) - 1)
        if left > right:
            left, right = right, left
        neighbour_state[left: right + 1] = reversed(neighbour_state[left: right + 1])
        return neighbour_state

    def actions(self, state):
        """ action that can be excuted in given state """
        return [self.two_opt]

    def result(self, state, action):
        """  result after applying the given action on the given state """
        return action(state)

    def path_cost(self, c, state1, action, state2):
        """ total distance for the Traveling Salesman to be covered if in state2  """
        cost = 0
        for i in range(len(state2) - 1):
            cost += distances[state2[i]][state2[i + 1]]
        cost += distances[state2[0]][state2[-1]]
        return cost

    def value(self, state):
        """ value of path cost given negative for the given state """
        return -1 * self.path_cost(None, None, None, state)

In [410]:
## Resolveremos el TSP para las ciudades de la lista de ciudades de Rumanía.
## ['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']

Esta es la imagen del mapa de Rumanía. 

![image.png](attachment:image.png)

In [411]:
# Usaremos la siguiente representacion del libro AIMA para el mapa de Rumanía.

romania_map = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

romania_map.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

Es bastante sencillo entender este `romania_map`. El primer nodo ** Arad ** tiene tres vecinos llamados ** Zerind **, ** Sibiu **, ** Timisoara **. Cada uno de estos nodos están a distancias 75, 140, 118 de ** Arad ** respectivamente. Y lo mismo ocurre con otros nodos.

Y `romania_map.locations` contiene las posiciones de cada uno de los nodos. 
Como heurística se puede usar la distancia en línea recta o la distancia manhattan (que es diferente de la proporcionada en `romania_map`) entre dos ciudades.

In [412]:
romania_locations = romania_map.locations
print(romania_locations)

{'Arad': (91, 492), 'Bucharest': (400, 327), 'Craiova': (253, 288), 'Drobeta': (165, 299), 'Eforie': (562, 293), 'Fagaras': (305, 449), 'Giurgiu': (375, 270), 'Hirsova': (534, 350), 'Iasi': (473, 506), 'Lugoj': (165, 379), 'Mehadia': (168, 339), 'Neamt': (406, 537), 'Oradea': (131, 571), 'Pitesti': (320, 368), 'Rimnicu': (233, 410), 'Sibiu': (207, 457), 'Timisoara': (94, 410), 'Urziceni': (456, 350), 'Vaslui': (509, 444), 'Zerind': (108, 531)}


In [413]:
# node colors, node positions and node label positions
node_positions = romania_map.locations
node_label_pos = { k:[v[0],v[1]-10]  for k,v in romania_map.locations.items() }
edge_weights = {(k, k2) : v2 for k, v in romania_map.graph_dict.items() for k2, v2 in v.items()}

romania_graph_data = {  'graph_dict' : romania_map.graph_dict,
                        'node_positions': node_positions,
                        'node_label_positions': node_label_pos,
                         'edge_weights': edge_weights
                     }

![image-2.png](attachment:image-2.png)

In [414]:
## el siguiente código crea un diccionario y calcula y añade al diccionario la distancia manhattan entre las ciudades. 
import numpy as np

distances = {}
all_cities = []

for city in romania_map.locations.keys():
    distances[city] = {}
    all_cities.append(city)
    
all_cities.sort()
print(all_cities)

for name_1, coordinates_1 in romania_map.locations.items():
        for name_2, coordinates_2 in romania_map.locations.items():
            distances[name_1][name_2] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])
            distances[name_2][name_1] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])

['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']


In [415]:
# Creamos una instancia del problema TSP con la lista de ciudades anterior que se na extraido del mapa.
# En el mapa hay informacion de las distancias que se utilizan en la clase TSP_problem para calcular el coste y las heurísticas.
tsp = TSP_problem(all_cities)

In [416]:
## Redefinimos el hill climbing de AIMA para que el método de generacion de vecinos sea acceder al grafo que hemos definido para el TSP
##  Escalada por máxima pendiente con 100 vecinos generados con el procedimiento 2-opt


def hill_climbing(problem):
    
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    
    def find_neighbors(state, number_of_neighbors=100):
        """ finds neighbors using two_opt method """
        
        neighbors = []
        
        for i in range(number_of_neighbors):
            new_state = problem.two_opt(state)
            neighbors.append(Node(new_state))
            state = new_state
            
        return neighbors

    # as this is a stochastic algorithm, we will set a cap on the number of iterations
    iterations = 10000
    
    current = Node(problem.initial)
    while iterations:
        neighbors = find_neighbors(current.state)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            current.state = neighbor.state
        iterations -= 1
        
    return current.state

In [417]:
# Y lo resolvemos con escalada por máxima pendiente. 
hill_climbing(tsp)

['Hirsova',
 'Mehadia',
 'Bucharest',
 'Timisoara',
 'Lugoj',
 'Drobeta',
 'Sibiu',
 'Zerind',
 'Arad',
 'Urziceni',
 'Pitesti',
 'Vaslui',
 'Giurgiu',
 'Oradea',
 'Eforie',
 'Fagaras',
 'Neamt',
 'Craiova',
 'Rimnicu',
 'Iasi']

### Ejercicio 1. Resuelve el problema TSP con el algoritmo de escalada por máxima pendiente en el mapa de ciudades de Rumanía y explica el resultado obtenido. 

Realiza varias ejecuciones y comenta razonadamente las propiedades del algoritmo: eficiencia y optimalidad en base a la ejecución.  ¿Ha encontrado el algoritmo el óptimo global? ¿Ha encontrado la misma solución en distintas ejecuciones?
Sólo se pide hacer una comparativa teórica (breve) con cómo se comporta este algoritmo.
Relacionarlo con otros algoritmos vistos en clase.


Obviamente el óptimo global no se ha encontrado, de hecho la solución aportada es bastante mala. Se queda estancado en un óptimo demasiado local. En distintas ejecuciones se encuentran distintas soluciones.

En el caso de la eficiencia, es mejor que los algoritmos de búsqueda y que refuerzo, pero perdemos mucha optimalidad. En comparación con los que veremos a continuación (enfriamiento simulado) es peor porque tiene un enfoque demasiado codicioso.

## Parte 2. Enfriamiento simulado ( simulated annealing) 
El algoritmo de enfriamiento simulado puede manejar las situaciones de óptimo local o mesetas típicas en algoritmos de escalada.
<br>
El enfriamiento simulado es bastante similar a la escalada pero en lugar de elegir el mejor movimiento en cada iteración, elige un movimiento aleatorio. Si este movimiento aleatorio nos acerca al óptimo global, será aceptado,
pero si no lo hace, el algoritmo puede aceptar o rechazar el movimiento en función de una probabilidad dictada por la temperatura.  Cuando la `temperatura` es alta, es más probable que el algoritmo acepte un movimiento aleatorio incluso si es malo. A bajas temperaturas, solo se aceptan buenos movimientos, con alguna excepción ocasional.
Esto permite la exploración del espacio de estado y evita que el algoritmo se atasque en el óptimo local.

    Usaremos la implementación de AIMA del modulo search.py
    
    def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice

Como hemos visto en clase hay varios métodos de enfriamiento (scheduling routine) 
Se puede variar el método de enfriamiento. En la implementación actual estamos usando el método de enfriamiento exponencial (que se pasa como parámetro). 

    def exp_schedule(k=20, lam=0.005, limit=100):
        """One possible schedule function for simulated annealing"""
        return lambda t: (k * math.exp(-lam * t) if t < limit else 0)

Como ejemplo, vamos a definir un problema sencillo de encontrar el valor más alto en una rejilla. Este problema está definido en el módulo search.py como PeakFindingProblem. Lo reproducimos aquí y creamos una rejilla simple.

In [418]:
initial = (0, 0)
grid = [[3, 7, 2, 8], [5, 2, 9, 1], [5, 3, 3, 1]]

In [419]:
# Pre-defined actions for PeakFindingProblem
directions4 = { 'W':(-1, 0), 'N':(0, 1), 'E':(1, 0), 'S':(0, -1) }
directions8 = dict(directions4) 
directions8.update({'NW':(-1, 1), 'NE':(1, 1), 'SE':(1, -1), 'SW':(-1, -1) })

class PeakFindingProblem(Problem):
    """Problem of finding the highest peak in a limited grid"""

    def __init__(self, initial, grid, defined_actions=directions4):
        """The grid is a 2 dimensional array/list whose state is specified by tuple of indices"""
        Problem.__init__(self, initial)
        self.grid = grid
        self.defined_actions = defined_actions
        self.n = len(grid)
        assert self.n > 0
        self.m = len(grid[0])
        assert self.m > 0

    def actions(self, state):
        """Returns the list of actions which are allowed to be taken from the given state"""
        allowed_actions = []
        for action in self.defined_actions:
            next_state = vector_add(state, self.defined_actions[action])
            if next_state[0] >= 0 and next_state[1] >= 0 and next_state[0] <= self.n - 1 and next_state[1] <= self.m - 1:
                allowed_actions.append(action)

        return allowed_actions

    def result(self, state, action):
        """Moves in the direction specified by action"""
        return vector_add(state, self.defined_actions[action])

    def value(self, state):
        """Value of a state is the value it is the index to"""
        x, y = state
        assert 0 <= x < self.n
        assert 0 <= y < self.m
        return self.grid[x][y]


In [420]:
problem = PeakFindingProblem(initial, grid, directions4)

In [421]:
# Lo resolvemos con enfriamiento simulado

solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

9

In [422]:
problem.value(simulated_annealing(problem))

2

In [423]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state

In [424]:
solution = problem.value(hill_climbing(problem))
solution

7

### Ejercicio 2.  Resuelve el problema anterior de encontrar el punto máximo en una rejilla. Comenta y razona los resultados obtenidos en distintas rejjillas con los algoritmos de enfriamiento simulado y escalada por máxima pendiente. 
 
 
Ejemplo de rejilla para pruebas

grid = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]


In [425]:
initial = (0, 0)
grid = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40], [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40], [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00], [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40], [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]

In [426]:
problem = PeakFindingProblem(initial, grid, directions4)

In [427]:
solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

11.2

In [428]:
solutions = {problem.value(simulated_annealing(problem)) for i in range(12)}
max(solutions)

6.5

In [429]:
cont =0
total=0
for i in range(1000):
    valor = problem.value(simulated_annealing(problem))
    if valor == 11.2:
        cont +=1
    total += valor
print("Valores optimos encontrados " + str(cont))
print("Media " + str(total/1000))

Valores optimos encontrados 30
Media 1.3366000000000022


Observamos que si ejecutamos el algoritmo menos veces no encuentra una solución tan óptima como cuando ejecutamos mas veces.
Hemos hecho un bucle que comprueba en cuantos casos encuentra ese máximo y para 1000 casos nos da una media de 28 ejecuciones donde
lo encuentra. Por otro lado hemos calculado la media de las soluciones y vemos que dista bastante de la solución óptima, es decir, en general no obtendremos un buen resultado

In [430]:
solution = problem.value(hill_climbing(problem))
solution

0.0

El algoritmo de escalada siempre da el mismo resultado, mientras que el de enfriamiento no. Esto se debe a que el de escalada ha encontrado un óptimo local y el otro depende de la temperatura.

## Parte 3. Algoritmos genéticos


Se define una clase ProblemaGenetico que incluye los elementos necesarios para la representación de un problema de optimización que se va a resolver con un algoritmo genético. Los elementos son los que hemos visto en clase:

 - genes: lista de genes usados en el genotipo de los estados.
 - longitud_individuos: longitud de los cromosomas
 - decodifica: función de obtiene el fenotipo a partir del genotipo.
 - fitness: función de valoración.
 - muta: función de mutación de un cromosoma 
 - cruza: función de cruce de un par de cromosomas

In [431]:
import random

In [432]:
class ProblemaGenetico(object):
        def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness,longitud_individuos):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma)
            return valoracion

En primer lugar vamos a definir una instancia de la clase anterior correspondiente al problema de optimizar (maximizar o minimizar) la función cuadrado x^2 en el conjunto de los números naturales menores que 2^{10}.
Se usa este ejemplo (del que sabemos la solución) para ver todos los elementos y poder observar el comportamiento del algoritmo genético. 

In [433]:
# Será necesaria la siguiente función que interpreta una lista de 0's y 1's como un número natural:  
# La siguiente función que interpreta una lista de 0's y 1's como
# un número natural:  

def binario_a_decimal(x):
    x=x[::-1]
    return sum(b*(2**i) for (i,b) in enumerate(x)) 

In [434]:
binario_a_decimal((1,1,1,0))

14

In [435]:
# En primer luegar usaremos la clase anterior para representar el problema de optimizar (maximizar o minimizar)
# la función cuadrado en el conjunto de los números naturales menores que
# 2^{10}.

# Tenemos que definir funciones de cruce, mutación y fitness para este problema.

def fun_cruzar(cromosoma1, cromosoma2):
    """Cruza los cromosomas por la mitad"""
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2]+cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2]+cromosoma1[l2//2:l1]
    return [cruce1,cruce2]

def fun_mutar(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        cromosoma[p] =  (cromosoma[p]+1)%2
    return cromosoma

def fun_fitness_cuad(cromosoma):
    """Función de valoración que eleva al cuadrado el número recibido en binario"""
    n = binario_a_decimal(cromosoma)**2
    return n

cuadrados = ProblemaGenetico([0,1],binario_a_decimal,fun_mutar, fun_cruzar, fun_fitness_cuad,10)

Una vez definida la instancia cuadrados que representa el problema genético, probar alguna de las funciones definidas en la clase anterior, para esta instancia concreta. Por ejemplo:

In [436]:
cuadrados.decodifica([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1125

1125

In [437]:
cuadrados.fitness([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1265625

1265625

In [438]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

In [439]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

In [440]:
cuadrados.cruza([1,0,0,0,1,1,0,0,1,0,1],[0,1,1,0,1,0,0,1,1,1])
# Posible salida: [[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

[[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

### Ejercicio 3

   - Definir una función poblacion_inicial(problema_genetico,tamaño), para definir una población inicial de un tamaño dado, para una instancia dada de la clase anterior ProblemaGenetico

sugerencia: usar random.choice

   - Definir una función de cruce que recibe una instancia de Problema_Genetico y una población de padres (supondremos que hay un número par de padres), obtiene la población resultante de cruzarlos de dos en dos (en el orden en que aparecen)

cruza_padres(problema_genetico,padres)

   - Definir la función de mutación que recibe una instancia de Problema_Genetico, una población y una probabilidad de mutación, obtiene la población resultante de aplicar operaciones de mutación a cada individuo llamando a la función muta definida para el problema genético.
muta_individuos(problema_genetico, poblacion, prob)

In [441]:
def decimal_to_binary(decimal, tam):
    l = list()
    for i in range(tam):
        if decimal % 2 == 0:
            l =[0] + l
        else:
            l= [1] +l 
        decimal //=2
    return l

In [442]:
decimal_to_binary(7,10)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

In [443]:
#Definimos la función población_inicial. Usamos la función decimal_to_binary y vamos añadiendo a la lista números aleatorios
#para inicializar la población inicial.

def poblacion_inicial(problema_genetico, size):
    nums = list(range(2**10))
    l=list()
    for j in range(size):
        l = l + [decimal_to_binary(random.choice(nums), problema_genetico.longitud_individuos)]

    return l

In [444]:
poblacion_inicial(cuadrados,10)

[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 [1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]]

In [445]:
def cruza_padres(problema_genetico,padres):
    l = []
    for i in range(len(padres)//2):
        l  = l + problema_genetico.cruza(padres[2*i], padres[(2*i)+1])
    return l

In [446]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

cruza_padres(cuadrados,p1)
# Posible salida
# [[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
#  [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#  [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#  [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
#  [0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
#  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]]

[[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

In [447]:
def muta_individuos(problema_genetico, poblacion, prob):
    # hay que llamar a  problema_genetico.muta(x,prob) para todos los individuos de la poblacion.
    for i in range(len(poblacion)): 
        poblacion[i] = problema_genetico.muta(poblacion[i],prob)
    return poblacion

In [448]:
muta_individuos(cuadrados,p1,0.5)
# Posible salida:
#  [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#   [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
#   [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
#   [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#   [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

[[1, 1, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 0, 1, 1, 0, 1]]

In [449]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

In [450]:
muta_individuos(cuadrados,p1,0.5)

[[1, 1, 0, 0, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 1, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 1, 1]]

Vamos a definir una función de selección mediante torneo de n individuos de una población.  
La función recibe como entrada:
 - una instancia de la clase ProblemaGenetico
 - una población
 - el número n de individuos que vamos a seleccionar
 - el número k de participantes en el torneo
 - un valor opt que puede ser o la función max o la función min (dependiendo de si el problema es de maximización o de minimización, resp.).

seleccion\_por\_torneo(problema_genetico,poblacion,n,k,opt) 

Usar random.sample para seleccionar k elementos de una secuencia. 
Por ejemplo, random.sample(population=[2,5,7,8,9], k=3) devuelve [7,5,8]. 

In [451]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

In [452]:
#Ejemplo
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,max)
# Posible salida: [[1, 1, 1, 1, 1, 0, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]]


[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [453]:
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,min)
# [[0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 1, 0, 1, 0]]

[[0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 1, 1, 0]]

In [454]:
# La siguiente función implementa una posibilidad para el algoritmo genético completo: 
# inicializa t = 0 
# Generar y evaluar la Población P(t)
# Mientras no hemos llegado al número de generaciones fijado:  t < nGen
#    P1 = Selección por torneo de (1-size)·p individuos de P(t)
#    P2 = Selección por torneo de (size·p) individuos de P(t)
#    Aplicar cruce en la población P2
#    P4 = Union de P1 y P3
#    P(t+1) := Aplicar mutación P4 
#    Evalua la población P(t+1) 
#    t:= t+1
        
# Sus argumentos son:
# problema_genetico: una instancia de la clase ProblemaGenetico con la representación adecuada del problema de optimización 
# que se quiere resolver.
# k: número de participantes en los torneos de selección.
# opt: max ó min, dependiendo si el problema es de maximización o de minimización. 
# nGen: número de generaciones (que se usa como condición de terminación)
# size: número de individuos en cada generación
# prop_cruce: proporción del total de la población que serán padres. 
# prob_mutación: probabilidad de realizar una mutación de un gen.
### fun_poblacion_inicial: funcion que genera la poblacion inicial

def algoritmo_genetico(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar,fun_poblacion_inicial):
    poblacion= fun_poblacion_inicial(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for i in range(ngen):
        clear_output()
        print("Generacion " + str(i))
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)
        

    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 


Necesitarás definir la función auxiliar nueva_generacion(problema_genetico,poblacion,n_padres,n_directos,prob_mutar) que dada una población calcula la siguiente generación.

In [455]:
#Definir la función nueva_generacion

def nueva_generacion(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    
    cruces =  cruza_padres(problema_genetico,padres1)
    
    generacion = padres2+cruces
    
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    
    return resultado_mutaciones



### Ejercicio 4.  Ejecutar el algoritmo genético anterior, para resolver el problema anterior (tanto en minimización como en maximización).  
Hacer una valoración de resultados y comentarios sobre el comportamiento del algoritmmo. 
En la resolución del problema hay que tener en cuenta que el algoritmo genético devuelve un par con el mejor fenotipo encontrado y su valoración.

In [456]:
algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.1, poblacion_inicial)
# Salida esperada: (0, 0)

Generacion 19


(34, 1156)

In [457]:
algoritmo_genetico(cuadrados,3,max,20,10,0.7,0.1, poblacion_inicial)
# Salida esperada: (1023, 1046529)

Generacion 19


(1017, 1034289)

##  El problema de la mochila 

Se plantea el típico problema de la mochila en el que dados n objetos de pesos conocidos pi y valor vi (i=1,...,n) hay que elegir cuáles se meten en una mochila que soporta un peso P máximo. La selección debe hacerse de forma que se máximice el valor de los objetos introducidos sin superar el peso máximo.

### Ejercicio 5

Se pide definir la representación del problema de la mochila usando genes [0,1] y longitud de los individuos n.
Los valores 1 ó 0 representan, respectivamente, si el objeto se introduce o no en la mochila Tomados de izquerda a derecha, a partir del primero que no cabe, se consideran  todos fuera de la mochila,independientemente del gen en su posición. De esta manera, todos los individuos representan candidatos válidos.

El numero de objetos n determina la longitud de los individuos de la población.
En primer lugar es necesario definir una función de decodificación de la mochila que recibe como entrada:
* un cromosoma (en este caso, una lista de 0s y 1s, de longitud igual a n_objetos) 
* n: número total de objetos de la mochila
* pesos: una lista con los pesos de los objetos
* capacidad: peso máximo de la mochila.
La función decodifica recibe (cromosoma, n, pesos, capacidad) y devuelve una lista de 0s y 1s que indique qué objetos están en la mochila y cuáles no (el objeto i está en la mochila si y sólo si en la posición i-ésima de la lista hay un 1). Esta lista se obtendrá a partir del cromosoma, pero teniendo en cuenta que a partir del primer objeto que no quepa, éste y los siguientes se consideran fuera de la mochila, independientemente del valor que haya en su correspondiente posición de cromosoma. 

In [458]:
def decodifica_mochila(cromosoma, n, pesos, capacidad):
    peso_en_mochila = 0
    l = []
    for i in range(n):
        if cromosoma[i] == 1 and peso_en_mochila + pesos[i] <= capacidad:
            l.append(1)
            peso_en_mochila += pesos[i]
        elif cromosoma[i]== 0 or peso_en_mochila + pesos[i] > capacidad:
            l.append(0)
    return l 

In [459]:
decodifica_mochila([1,1,1,1,1], 5, [2,3,4,5,1], 5)

[1, 1, 0, 0, 0]

Para definir la función de evaluación (fitness) necesitamos calcular el valor total de los objetos que están dentro de la mochila que representa el cromosoma según la codificación utilizada en la función anterior. 

Se pide la función fitness (cromosoma, n_objetos, pesos, capacidad, valores) donde los parámetros son los mismos que en la función anterior, y valores es la lista de los valores de cada objeto

fitness(cromosoma, n_objetos, pesos, capacidad, valores)

Ejemplo de uso:
   fitness([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])
   7

In [460]:
def fitness_mochila(cromosoma, n_objetos, pesos, capacidad, valores):
    return valor

In [461]:
fitness_mochila([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])

8.5

Damos tres instancias concretas del problema de la mochila. Damos también sus soluciones optimas, para que se puedan comparar con los resultados obtenidos por el algoritmo genético:

In [462]:
# Problema de la mochila 1:
# 10 objetos, peso máximo 165
pesos1 = [23,31,29,44,53,38,63,85,89,82]
valores1 = [92,57,49,68,60,43,67,84,87,72]

# Solución óptima= [1,1,1,1,0,1,0,0,0,0], con valor 309

In [463]:
# Problema de la mochila 2:
# 15 objetos, peso máximo 750

pesos2 = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
valores2 = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]

# Solución óptima= [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1] con valor 1458

In [464]:
# Problema de la mochila 3:
# 24 objetos, peso máximo 6404180
pesos3 = [382745,799601,909247,729069,467902, 44328,
       34610,698150,823460,903959,853665,551830,610856,
       670702,488960,951111,323046,446298,931161, 31385,496951,264724,224916,169684]
valores3 = [825594,1677009,1676628,1523970, 943972,  97426,
       69666,1296457,1679693,1902996,
       1844992,1049289,1252836,1319836, 953277,2067538, 675367,
       853655,1826027, 65731, 901489, 577243, 466257, 369261]

# Solución óptima= [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1] con valoración 13549094

### Ejercicio 5

Se pide definir la representación del problema de la mochila usando genes [0,1] y longitud de los individuos n.
Los valores 1 ó 0 representan, respectivamente, si el objeto se introduce o no en la mochila Tomados de izquerda a derecha, a partir del primero que no cabe, se consideran  todos fuera de la mochila,independientemente del gen en su posición. De esta manera, todos los individuos representan candidatos válidos.

El numero de objetos n determina la longitud de los individuos de la población.
En primer lugar es necesario definir una función de decodificación de la mochila que recibe como entrada:
* un cromosoma (en este caso, una lista de 0s y 1s, de longitud igual a n_objetos) 
* n: número total de objetos de la mochila
* pesos: una lista con los pesos de los objetos
* capacidad: peso máximo de la mochila.
La función decodifica recibe (cromosoma, n, pesos, capacidad) y devuelve una lista de 0s y 1s que indique qué objetos están en la mochila y cuáles no (el objeto i está en la mochila si y sólo si en la posición i-ésima de la lista hay un 1). Esta lista se obtendrá a partir del cromosoma, pero teniendo en cuenta que a partir del primer objeto que no quepa, éste y los siguientes se consideran fuera de la mochila, independientemente del valor que haya en su correspondiente posición de cromosoma. 

In [465]:
def decodifica_mochila(cromosoma, n, pesos, capacidad):
    peso_en_mochila = 0
    l = []
    for i in range(n):
        if cromosoma[i] == 1 and peso_en_mochila + pesos[i] <= capacidad:
            l.append(1)
            peso_en_mochila += pesos[i]
        elif cromosoma[i]== 0 or peso_en_mochila + pesos[i] > capacidad:
            l.append(0)
    return l 

In [466]:
decodifica_mochila([1,1,1,1,1], 5, [2,3,4,5,1], 5)

[1, 1, 0, 0, 0]

Para definir la función de evaluación (fitness) necesitamos calcular el valor total de los objetos que están dentro de la mochila que representa el cromosoma según la codificación utilizada en la función anterior. 

Se pide la función fitness (cromosoma, n_objetos, pesos, capacidad, valores) donde los parámetros son los mismos que en la función anterior, y valores es la lista de los valores de cada objeto

fitness(cromosoma, n_objetos, pesos, capacidad, valores)

Ejemplo de uso:
   fitness([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])
   7

In [467]:
def fitness_mochila(cromosoma, n_objetos, pesos, capacidad, valores):
    mochila = decodifica_mochila(cromosoma, n_objetos, pesos, capacidad)
    valor = 0
    for i in range(n_objetos):
        if mochila[i] == 1:
            valor += valores[i] 
    return valor

In [468]:
fitness_mochila([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])

7

Damos tres instancias concretas del problema de la mochila. Damos también sus soluciones optimas, para que se puedan comparar con los resultados obtenidos por el algoritmo genético:

In [469]:
# Problema de la mochila 1:
# 10 objetos, peso máximo 165
pesos1 = [23,31,29,44,53,38,63,85,89,82]
valores1 = [92,57,49,68,60,43,67,84,87,72]



# Solución óptima= [1,1,1,1,0,1,0,0,0,0], con valor 309

In [470]:
def fitness_mochila_1(cromosoma):
    v = fitness_mochila(cromosoma, 10, pesos1, 165, valores1)
    return v
def decodifica_mochila_1(cromosoma):
    v = decodifica_mochila(cromosoma, 10, pesos1, 165)
    return v
m1g = ProblemaGenetico([0,1], decodifica_mochila_1, fun_mutar, fun_cruzar, fitness_mochila_1,10)

In [471]:
algoritmo_genetico(m1g,3,max,20,10,0.7,0.1, poblacion_inicial)

Generacion 19


([1, 1, 1, 1, 0, 1, 0, 0, 0, 0], 309)

In [472]:
# Problema de la mochila 2:
# 15 objetos, peso máximo 750

pesos2 = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
valores2 = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]

# Solución óptima= [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1] con valor 1458

In [473]:
def fitness_mochila_2(cromosoma):
    v = fitness_mochila(cromosoma, 15, pesos2, 750, valores2)
    return v
def decodifica_mochila_2(cromosoma):
    v = decodifica_mochila(cromosoma, 14, pesos2, 750)
    return v
m2g = ProblemaGenetico([0,1], decodifica_mochila_2, fun_mutar, fun_cruzar, fitness_mochila_2,15)

In [474]:
algoritmo_genetico(m2g,5,max,30,200,0.7,0.2, poblacion_inicial)

Generacion 29


([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1], 1453)

In [ ]:
Observamos que el valor de la solución obtenida con el algoritmo es muy parecido a la solución óptima

In [475]:
# Problema de la mochila 3:
# 24 objetos, peso máximo 6404180
pesos3 = [382745,799601,909247,729069,467902, 44328,
       34610,698150,823460,903959,853665,551830,610856,
       670702,488960,951111,323046,446298,931161, 31385,496951,264724,224916,169684]
valores3 = [825594,1677009,1676628,1523970, 943972,  97426,
       69666,1296457,1679693,1902996,
       1844992,1049289,1252836,1319836, 953277,2067538, 675367,
       853655,1826027, 65731, 901489, 577243, 466257, 369261]

# Solución óptima= [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1] con valoración 13549094

In [476]:

def fitness_mochila_3(cromosoma):
    v = fitness_mochila(cromosoma, 24, pesos3,6404180 , valores3)
    return v
def decodifica_mochila_3(cromosoma):
    v = decodifica_mochila(cromosoma, 24, pesos3, 6404180)
    return v
m3g = ProblemaGenetico([0,1], decodifica_mochila_3, fun_mutar, fun_cruzar, fitness_mochila_3,24)


In [477]:
algoritmo_genetico(m3g,5,max,40,300,0.7,0.2, poblacion_inicial)

Generacion 39


([0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0],
 13419200)

In [ ]:
En este caso, seguimos observando que la solución obtenida con el algoritmo es muy parecida a la solución óptima

### Ejercicio 6

Definir variables m1g, m2g y m3g, referenciando a instancias de Problema_Genetico que correspondan, respectivamente, a los problemas de la mochila anteriores. Resuelve los problemas y comentar los resultados obtenidos en cuanto a eficiencia y calidad de los resultados obtenidos.

Algunas de las salidas posibles variando los parámetros.

In [478]:
def algoritmo_genetico_t(x,y,z,w,s,t,p):
    return algoritmo_genetico(x,y,z,w,s,t,p,poblacion_inicial)

algoritmo_genetico_t(m1g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 1, 0, 1, 0, 0, 0, 0], 309)

Generacion 99


([1, 1, 1, 1, 0, 1, 0, 0, 0, 0], 309)

In [479]:
algoritmo_genetico_t(m2g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0], 1444)

Generacion 99


([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1], 1449)

In [480]:

algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0], 1439)

Generacion 199


([0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1], 1432)

In [481]:

algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1], 1458)

Generacion 199


([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1], 1449)

In [482]:

algoritmo_genetico_t(m3g,5,max,400,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], 13518963)

Generacion 399


([0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1],
 13432502)

observamos que los resultados obtenidos son muy parecidos y cercanos a los esperados, por tanto , las conclusiones son que el algoritmo funciona bien

In [484]:
#Estos tardaban mucho, comentalos como si saliera un resultado cercano pero no exacto el suyo. 
#Según la teoría este y el siguiente deberían de dar casi lo mismo porque llega un punto
#que aumentar las iteraciones no cambia casi, y los otros incluso peores por eso mismo, porque no
#cambia demasiado hacer más iteraciones, sino un nùmero decente y una población inicial decente.


algoritmo_genetico_t(m3g,4,max,600,200,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], 13524340)

In [ ]:
algoritmo_genetico_t(m3g,4,max,1000,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13449995)

In [485]:
algoritmo_genetico_t(m3g,3,max,1000,100,0.75,0.1)
# ([1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13412953)

Generacion 999


([1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0],
 13456545)

In [ ]:
algoritmo_genetico_t(m3g,3,max,2000,100,0.75,0.1)
# ([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 13366296)

In [ ]:
algoritmo_genetico_t(m3g,6,max,2000,100,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1], 13549094)

### Ejercicio 7
Resolver mediante una configuración de un algoritmo genético el problema del cuadrado mágico que consiste en colocar en un cuadrado n × n los números naturales de 1 a n^2, 
de tal manera que las filas, las columnas y las diagonales principales sumen los mismo. 
Ejemplo: una solucion para n= 3
    
    4 3 8
    9 5 1
    2 7 6
    
- Dimensionn del cuadrado: N
- Suma común: SUMA=(N·(N^2 + 1))/2
    
    Comenta el resultado y el rendimiento del algoritmo para distintos parámetros.
    

    
    

In [489]:
"""
El cuadrado lo representamos como un array lineal de tamaño n*n
"""


from math import sqrt, floor

"""
La función fitness del cuadrado simplemente suma las diferencias de cada fila, columna
y diagonal con respecto del valor esperado e intentamos minimizar esa diferencia.
"""

def fitnessMagic(cromosoma):
    n  = floor(sqrt(len(cromosoma)))
    magico_obj = (n*(n**2+1))//2
    diferencias = 0
    
    magico_fila = 0
    magico_columna = 0 
    magico_diag_arriba = 0
    magico_diag_abajo = 0
    
    for i in range(n):
        magico_fila = 0
        magico_columna = 0
        for j in range(n):
            if i==j:magico_diag_arriba += cromosoma[n*i+j]
            if i== (n-j-1): magico_diag_abajo += cromosoma [n*i+j]
            
            magico_fila += cromosoma[n*i+j]
            magico_columna += cromosoma[n*j+i]
            
        diferencias += abs (magico_fila - magico_obj) + abs(magico_columna - magico_obj)
        
    diferencias +=  abs(magico_diag_arriba - magico_obj) + abs(magico_diag_abajo - magico_obj)
    return diferencias



"""
Esta es una implementación del cruce PMX estudiado en clase,
para ello elegimos dos puntos y vamos haciendo el procedimiento ayudándonos
de un array de indices. Si no estás en la solución entonces te coloco y si no voy cogiendo los siguientes que
aún no estén elegidos.

"""

def PMX(cromosoma1, cromosoma2):  

    
    tamano =  len(cromosoma1)
    
    
    cruce1= random.randint(0, size)
    cruce2 = random.randint(0, size - 1)
    if cruce2 >= cruce1:
        cruce2 += 1
    else:  
        cruce2, cruce1 = cruce1, cruce2
    
    
    crom1_m = cromosoma1[cruce1:cruce2]
    crom2_m = cromosoma2[cruce1:cruce2]
    
    cromosoma1 = cromosoma1[:cruce1] + crom2_m + cromosoma1[cruce2:]
    cromosoma2 = cromosoma2[:cruce1] + crom1_m + cromosoma2[cruce2:] 
    
    mapeo1 = cromosoma1[cruce1:cruce2]
    mapeo2 = cromosoma2[cruce1:cruce2]
    for i in (list(range(0,cruce1)) + list(range(cruce2,tamano))):
                while cromosoma1.count(cromosoma1[i])>1:
                    indice_mapa = mapeo1.index(cromosoma1[i])
                    cromosoma1[i] = mapeo2[indice_mapa]
                
                while cromosoma2.count(cromosoma2[i])>1:
                    indice_mapa = mapeo2.index(cromosoma2[i])
                    cromosoma2[i] = mapeo1[indice_mapa]
    
    return [cromosoma1, cromosoma2]

def mutarMagic(cromosoma,prob):
    """Elige dos elemento al azar del cromosoma y los intercambia con una probabilidad igual a prob"""
    l = len(cromosoma)
    p1 = random.randint(0,l-1)
    p2 = random.randint(0, l-1)
    if prob > random.uniform(0,1):
        cromosoma[p1] , cromosoma[p2] = cromosoma[p2], cromosoma[p1]
    return cromosoma

"""
Otra posibilidad de mutar, cogiendo una fila y le hacemos un shuffle
pero da peores resultados

def mutarMagic(cromosoma,prob):
    
    l = len(cromosoma)
    p1 = random.randint(0,floor(sqrt(l)))
    if prob > random.uniform(0,1):
        cromosoma_medio = cromosoma[p1:(p1+floor(sqrt(l)))]
        random.shuffle(cromosoma_medio)
        cromosoma = cromosoma[:p1]+cromosoma_medio + cromosoma[(p1+floor(sqrt(l))):]
    return cromosoma
"""

#Simplemente hacer un shuffle del array [1,2,3,4,5,6,7,8,9,10...]

def poblacion_inicial(problema_genetico, size):
    l = list()
    tam_cromosoma = problema_genetico.longitud_individuos
    for i in range(size):
            elementos = list(range(1, tam_cromosoma+1))
            random.shuffle(elementos)
            l = l + [elementos]

    return l




        
k=4        
ngen= 30
size = 3**2
prop_cruces = 0.7
prob_mutar = 0.1
        
problemaMagic = ProblemaGenetico(list(range(1,size+1)),lambda x:x,mutarMagic, PMX, fitnessMagic,size)
        
    
tam_poblacion = 150
algoritmo_genetico(problemaMagic,k,min,ngen,tam_poblacion,prop_cruces,prob_mutar,poblacion_inicial)

Generacion 29


([8, 3, 4, 1, 5, 9, 6, 7, 2], 0)

In [491]:
k=5        
ngen= 30
size = 3**2
prop_cruces = 0.7
prob_mutar = 0.1

        
problemaMagic = ProblemaGenetico(list(range(1,size+1)),lambda x:x,mutarMagic, PMX, fitnessMagic,size)
        
    
tam_poblacion = 150
algoritmo_genetico(problemaMagic,k,min,ngen,tam_poblacion,prop_cruces,prob_mutar,poblacion_inicial)

Generacion 29


([2, 7, 6, 9, 5, 1, 4, 3, 8], 0)

In [497]:
k=7        
ngen= 100
size = 5**2
prop_cruces = 0.7
prob_mutar = 0.1

        
problemaMagic = ProblemaGenetico(list(range(1,size+1)),lambda x:x,mutarMagic, PMX, fitnessMagic,size)
        
    
tam_poblacion = 100
algoritmo_genetico(problemaMagic,k,min,ngen,tam_poblacion,prop_cruces,prob_mutar,poblacion_inicial)

Generacion 99


([21,
  23,
  9,
  2,
  10,
  16,
  5,
  6,
  20,
  17,
  7,
  19,
  13,
  11,
  15,
  3,
  4,
  12,
  25,
  22,
  18,
  14,
  24,
  8,
  1],
 4)

El algoritmo, gracias principalmente al cruce PMX, obtiene unos resultados buenos y converge a un valor prácticamente 0, o incluso llegando al óptimo, incluso para cuadrados de tamaño 5x5 o 6x6. (En 3x3 prácticamente
siempre alcanza la solución)

Esta función de cruce consigue mantener información de cromosomas buenos al no perder el orden en que se encontraban, que es
realmente importante en el caso del cuadrado mágico. Un cruce en orden también habría sido positivo en este problema.

La mutación es sencilla y tampoco influye demasiado. Simplemente cambiamos dos valores del cuadrado. 

Vemos que el torneo de entre 4 y 7 dan un buen resultado pues si fuera más alto obtendríamos un resultado demasiado
codicioso y un cambio en el caso del cuadrado puede alterar radicalmente la solución (enfoque greedy es malo).

### Ejercicio 8. Formación de equipos de trabajo para los TFG en la Facultad de Informática.
Se os da el enunciado de forma separada. 

In [ ]:
from math import floor,ceil,sqrt
import random

"""
Función fitness del problema

El fitness estaba dividido en tres partes
1º Calculamos el fitness de los grupos formados y los grupos esperados con la formula del boletín
    Si la división no es exacta tomamos un valor flotante para los grupos esperados
    
2º Calculamos con una forma similar el fitness de la relación entre alumnos por grupo y alumnos esperados por grupo.
    1-abs(n_alumnos_en_el_grupo - n_esperados_por_grupo)/(n_alumnos - n_por_grupo) para cada grupo
    y lo dividimos entre el numero de grupos

3º Calculamos la desviación tipica en el array de cantidad de roles por grupo y sumamos 1/(1+stdev) al fitness por
cada grupo, haciendo después la media. Esto penaliza que haya variaciones grandes en el número de roles por grupo 

Pero teníamos dos problemas: La función stdev solo reflejaba la variación de un grupo concreto (podía tener muchos
alumnos pero estar bien repartido, pero eso no es información tan relevante) y además era bastante costosa

El otro estaba relacionado con el fitness de cantidad de grupos formados, que complicaba el problema pues ir cambiando
la cantidad de grupos dificultaba que las otras dos medidas fueran representativas y el problema no convergía decentemente.

Por tanto lo hemos cambiado de esta forma:

Los cromosomas tienen siempre el número de grupos adecuado aunque no distribuidos uniformemente por alumnos.

La parte de función de fitness asociada a los roles simplemente calcula para cada equipo cuanto de lejos están sus roles
del valor estimado de rol por grupo (n_por_grupo / n_roles, puede ser no entero sin ningún problema) y hace
un cálculo similar al del n_alumnos_grupo

"""

def fitnessRoles(n_por_grupo, roles_alumno, n_roles, cromosoma):
    n_alumnos = len(cromosoma)
    n_grupos= (ceil(n_alumnos/n_por_grupo)) 
    
    '''
    Intentamos hacer un código que funcionara para un valor variable de grupos, pero no obteníamos resultados
    aceptables para el resto de la configuración.
    
    n_grupos_esperado = n_alumnos/n_por_grupo
    fitness_n_grupos = 1-abs(n_grupos - n_grupos_esperado)/(n_alumnos- n_grupos_esperado)
    
    '''

    
    fitness_n_alumnos=0
    fitness_roles=0
    
    conjuntos_grupo = [dict() for _  in range(n_grupos)]
    contador = dict()
   
    
    for i, grupo in enumerate(cromosoma):
        contador[grupo] = contador.get(grupo,0)+1
        valor = conjuntos_grupo[grupo].get(roles_alumno[i], 0)
        conjuntos_grupo[grupo][roles_alumno[i]]= valor+1
    

    alumnos_por_rol = n_por_grupo / n_roles
    
    
    for i,grupo in enumerate(conjuntos_grupo):
            n_alumnos_grupo = contador.get(i,0)
            fitness_n_alumnos+= 1-abs(n_alumnos_grupo - n_por_grupo)/(n_alumnos - n_por_grupo)
            valores = list(grupo.values())
            long_roles = len(valores)
            
            for j in valores:
                fitness_roles += 1-abs(j - alumnos_por_rol)/(n_por_grupo - alumnos_por_rol)
                
            if(long_roles < n_roles):
                fitness_roles += (1-abs(alumnos_por_rol)/(n_por_grupo - alumnos_por_rol))*(n_roles - long_roles)
    
    fitness_n_alumnos = fitness_n_alumnos / n_grupos 
    fitness_roles = fitness_roles / (n_grupos*n_roles)
    return fitness_n_alumnos + fitness_roles




"""


Generamos un vector con valores aleatorios de 1 a n donde n es el número de grupo esperado
esto lo hacemos tantas veces como tamaño de la población

Originalmente generábamos un valor aleatorio de grupos también pero como explicamos anteriormente
la convergencia del algoritmo era muy mala si variábamos esto.

"""
def poblacionInicial(problema_genetico, size, n_por_grupo):
    n_cromosoma = problema_genetico.longitud_individuos
    n_grupos = ceil(n_cromosoma / n_por_grupo)
    poblacion = list()
    for k in range(size):
        x = [random.randint(0, n_grupos-1) for i in range(n_cromosoma)] 
        poblacion.append(x)
    return poblacion






def fun_cruzar_dos_puntos(cromosoma1, cromosoma2):
    """Cruce de dos puntos estudiado en clase"""
    l1 = len(cromosoma1)
    p1= random.randint(0, l1)
    p2 = random.randint(0, l1 - 1)
    if p2 >= p1:
        p2 += 1
    else:  
        p2, p1 = p1, p2
    
    cruce1 = cromosoma1[0:p1]+cromosoma2[p1:p2]+ cromosoma1[p2:l1]
    cruce2 = cromosoma2[0:p1]+cromosoma1[p1:p2]+ cromosoma2[p2:l1]
    return [cruce1,cruce2]




def fun_cruzar_uniforme(cromosoma1, cromosoma2):
    """Cruce de varios puntos en función de una probabilidad"""
    l1 = len(cromosoma1)
    for i in range(l1):
        p = random.uniform(0,1)
        if p >= 0.5:
            cromosoma1[i],cromosoma2[i]= cromosoma2[i],cromosoma1[i]
    return [cromosoma1, cromosoma2]



def mutarRoles(cromosoma,prob, n_grupos):
    """Elige elementos al azar del cromosoma y los cambia de grupo. Favorece el número de grupos"""
    l = len(cromosoma)
    while prob > random.uniform(0,1):
        p1 = random.randint(0,l-1)
        cromosoma[p1] = random.randint(0, n_grupos-1)
    return cromosoma




        
k=10   
ngen= 20
size = 2000
prop_cruces = 0.7
prob_mutar = 0.1
n_por_grupo=4
n_roles = 4

roles = [0,1,2,3]*500
random.shuffle(roles)
        
problemaRoles = ProblemaGenetico(list(range(0,ceil(size/n_por_grupo))),lambda x:x,lambda crom, prob:mutarRoles(crom, prob, ceil(size/n_por_grupo)), fun_cruzar_dos_puntos, lambda x:fitnessRoles(n_por_grupo,roles, n_roles, x),size)
tam_poblacion = 100
algoritmo_genetico(problemaRoles,k,max,ngen,tam_poblacion,prop_cruces,prob_mutar,lambda x,y:poblacionInicial(x,y,n_por_grupo))
    

In [ ]:
k=10   
ngen= 20
size = 2000
prop_cruces = 0.7
prob_mutar = 0.1
n_por_grupo=4
n_roles = 4

roles = [0,1,2,3]*500
random.shuffle(roles)
        
problemaRoles = ProblemaGenetico(list(range(0,ceil(size/n_por_grupo))),lambda x:x,lambda crom, prob:mutarRoles(crom, prob, ceil(size/n_por_grupo)), fun_cruzar_uniforme, lambda x:fitnessRoles(n_por_grupo,roles, n_roles, x),size)
tam_poblacion = 100
algoritmo_genetico(problemaRoles,k,max,ngen,tam_poblacion,prop_cruces,prob_mutar,lambda x,y:poblacionInicial(x,y,n_por_grupo))
    

In [ ]:
def mutarRoles_cambio(cromosoma,prob):
    """Elige dos elementos del cromosoma y los cambia de grupo. Favorece los roles equilibrados"""
    l = len(cromosoma)
    p1 = random.randint(0,l-1)
    p2 = random.randint(0,l-1)
    cromosoma[p1], cromosoma[p2] =  cromosoma[p2], cromosoma[p1]
    return cromosoma

k=10   
ngen= 20
size = 2000
prop_cruces = 0.7
prob_mutar = 0.1
n_por_grupo=4
n_roles = 4

roles = [0,1,2,3]*500
random.shuffle(roles)
        
problemaRoles = ProblemaGenetico(list(range(0,ceil(size/n_por_grupo))),lambda x:x,mutarRoles_cambio, fun_cruzar_dos_puntos, lambda x:fitnessRoles(n_por_grupo,roles, n_roles, x),size)
print(fun_cruzar_uniforme([0,1,2,3,4,5],[11,12,13,14,15,16]))
tam_poblacion = 100
algoritmo_genetico(problemaRoles,k,max,ngen,tam_poblacion,prop_cruces,prob_mutar,lambda x,y:poblacionInicial(x,y,n_por_grupo))
    

Hemos intentado aplicar varias funciones de cruce y varios parámetros. Por ejemplo, el cruce de dos puntos nos da una solución mejor que el cruce uniforme basa en probabilidad en término medio. Y aumentar demasiado el k de los torneos, pese a ser una muestra grande, no parece favorecer la variedad en los cromosomas. La mutación de intercambio ofrece una solución mejor para obtener una buena tasa de roles equilibrados mientras que la mutación de asignar a un alumno otro grupo directamente favorece más bien la tasa de cantidad de alumnos equilibrada entre los grupos.

Las probabilidades de mutar no pueden ser demasiado altas pues se pierde información de los cromosomas originales, y 0.1 ha sido un valor decente que ha fomentado la mejora de manera regular.

Nuestra función de fitness alcanza como mucho el valor 2 (hasta 1 en los alumnos bien repartidos y hasta 1 buen número de roles extra) aunque en caso de divisiones no enteras, este valor 2 será imposible de alcanzar, pero intentará llegar a lo máximo posible.

También planteamos ponderar uno u otro de los valores del fitness (darle más importancia a que los alumnos estuvieran bien repartidos o a que haya un número apropiado de alumnos por grupo) pero no aportaba mucho. Al final se enfocaba demasiado en uno o en otro o si la ponderación era baja, apenas afectaba a la solución.

Para casos pequeños (y no el de 2000) sí que conseguimos resultados buenos, incluso óptimos, aquí sin embargo pese a una población alta, el problema se estanca, obteniendo un reparto de alumnos decente pero no tan cercano al 2 de fitness que andamos buscando.

Tal vez con otro planteamiento se podría haber mantenido mejor la información entre genes, como usando conjuntos que hagan referencia a los alumnos de un grupo y olvidarnos del orden en el array. 

Sin embargo, con todo esto, obtenemos una mejora en las sucesivas iteraciones lo que nos dice que el algoritmo al menos funciona.